# Chat with your CSV documents!

The following notebook illustrates how you can use the Semantic Kernel to create
a basic chat experience over your CSV documents.

It primarily makes use of embeddings and the Semantic Kernel's concept of Memory. 

In [ ]:
!python -m pip install semantic-kernel==0.2.9.dev

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, OpenAITextEmbedding

import pickle 
import pandas as pd
from IPython.display import clear_output

### Load your CSV 
The data included is a curated list of drawing tutorials for people wanting to learn how to draw
different animated characters, animals, and people. 

In [ ]:
df = pd.read_csv("../data/art-tutorial-links.csv")

In [ ]:
df.head()

### Setup the Semantic Kernel

In [ ]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))
kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

### Create embeddings using the appropriate columns in the CSV

In [ ]:
# print("Adding rows of the CSV to VolatileMemory.");
# memory_collection_name = "csv-file"
# for idx, row in df.iterrows():
#     clear_output(wait=True)
#     await kernel.memory.save_reference_async(
#         collection=memory_collection_name,
#         description=row['Description'],
#         text=str(row['Description']+ " | " + row['Difficulty']),
#         external_id=row["Link"],
#         external_source_name=memory_collection_name
#     )
#     print("Row {} saved".format(idx))

### Persist kernel memory using pickle

In [ ]:
# temp = kernel.memory._storage._store
# with open("kernel_memory_storage_store.pkl", "wb") as f:
#     pickle.dump(kernel.memory._storage._store, f)

with open("kernel_memory_storage_store.pkl", "rb") as f:
    kernel.memory._storage._store = pickle.load(f)

In [ ]:
# print("Begin chatting (type 'exit' to exit):\n")
# chatting = True
# while chatting:
#     chatting = await chat(kernel, chat_func, context)

### Chat to get the best Youtube link!

In [ ]:
memory_collection_name = "csv-file"

In [ ]:
# ask = "I love Batman! I want an easy tutorial for how to draw him!"
#ask = "I want to learn how to draw animals for beginners"
ask = "I want to learn how to draw comic book characters"
print("===========================\n" + "Query: " + ask + "\n")

memories = await kernel.memory.search_async(memory_collection_name, ask, limit=5, min_relevance_score=0.77)

for i, memory in enumerate(memories):
    print("Result {}".format(i+1))
    print("  URL:     : " + memory.id)
    print("  Description    : " + memory.description)
    print("  Relevance: " + str(memory.relevance))
    print()
    

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo(memory.id.split("?v=")[1], width=800, height=600)